In [ ]:
# Install required packages with compatible versions
!pip install "langchain<0.1.0" langchain-community langchain-google-genai chromadb sentence-transformers pypdf gradio

In [ ]:
# Import libraries
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
import os
import tempfile


In [ ]:
#Global variables
vectorstore = None
qa_chain = None

def process_pdf(pdf_file, api_key):
    """Process uploaded PDF and create vector store"""
    global vectorstore, qa_chain

    try:
        # Validate inputs
        if not api_key or len(api_key.strip()) == 0:
            return "❌ Please enter a valid API key"

        if pdf_file is None:
            return "❌ Please upload a PDF file"

        # Set API key
        os.environ["GOOGLE_API_KEY"] = api_key.strip()

        # Handle file path (Gradio can pass either bytes or path)
        if isinstance(pdf_file, bytes):
            # Save uploaded file temporarily
            with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
                tmp_file.write(pdf_file)
                tmp_path = tmp_file.name
        else:
            # File is already a path
            tmp_path = pdf_file

        # Load PDF
        loader = PyPDFLoader(tmp_path)
        documents = loader.load()

        if len(documents) == 0:
            return "❌ No content found in PDF"

        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        texts = text_splitter.split_documents(documents)

        # Create embeddings
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Create vector store
        vectorstore = Chroma.from_documents(
            documents=texts,
            embedding=embeddings
        )

        # Set up Gemini LLM - Fixed configuration
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.7,
            google_api_key=api_key.strip(),
            convert_system_message_to_human=True  # This is the key fix!
        )

        # Create RAG prompt template (compatible with RetrievalQA)
        prompt_template = """You are an assistant for question-answering tasks.
                            Use the following pieces of retrieved context to answer the question.
                            If you don't know the answer, say that you don't know.
                            Keep the answer concise.

                            Context:
                            {context}

                            Question: {question}

                            Answer:"""

        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )

        # Create QA chain with custom prompt
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )

        # Clean up temp file if we created one
        if isinstance(pdf_file, bytes):
            os.unlink(tmp_path)

        return f"✅ PDF processed successfully! Loaded {len(documents)} pages with {len(texts)} chunks. You can now ask questions."

    except Exception as e:
        return f"❌ Error processing PDF: {str(e)}"

def chat(message, history):
    """Handle chat messages"""
    global qa_chain

    if qa_chain is None:
        return history + [[message, "⚠️ Please upload and process a PDF first!"]]

    if not message or len(message.strip()) == 0:
        return history

    try:
        result = qa_chain({"query": message})
        return history + [[message, result["result"]]]
    except Exception as e:
        return history + [[message, f"❌ Error: {str(e)}"]]

# Create Gradio interface
with gr.Blocks(title="RAG PDF Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📚 RAG PDF Chatbot with Gemini")
    gr.Markdown("Upload a PDF and chat with it using Google's Gemini AI")

    with gr.Row():
        with gr.Column(scale=1):
            api_key_input = gr.Textbox(
                label="🔑 Google API Key",
                type="password",
                placeholder="Enter your Gemini API key (starts with AIza...)"
            )
            pdf_input = gr.File(
                label="📄 Upload PDF",
                file_types=[".pdf"]
            )
            upload_btn = gr.Button("🚀 Process PDF", variant="primary")
            status_output = gr.Textbox(label="📊 Status", interactive=False)

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=400, label="💬 Chat")
            msg = gr.Textbox(
                label="Ask a question about your PDF",
                placeholder="Type your question here and press Enter..."
            )
            clear = gr.Button("🗑️ Clear Chat")

    # Event handlers
    upload_btn.click(
        fn=process_pdf,
        inputs=[pdf_input, api_key_input],
        outputs=status_output
    )

    msg.submit(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=chatbot
    ).then(
        lambda: "",
        outputs=msg
    )

    clear.click(lambda: [], None, chatbot, queue=False)

    gr.Markdown("""
    ### 📖 How to use:
    1. Get a free API key from [Google AI Studio](https://aistudio.google.com/app/apikey)
    2. Enter your API key above (keep it safe!)
    3. Upload a PDF document
    4. Click '🚀 Process PDF' and wait for confirmation
    5. Start asking questions about your PDF!

    ### ⚡ Features:
    - RAG (Retrieval-Augmented Generation) for accurate answers
    - Gemini 1.5 Flash for fast responses
    - Free HuggingFace embeddings
    - Processes PDFs of any size
    - Using LangChain <0.1.0 for maximum compatibility
    """)

# Launch the app
demo.launch(debug=True, share=True)